In [ ]:
library(tidyverse)
library(dagitty)

In [ ]:
df = readr::read_csv("simulated_data_2.csv") |> 
    mutate(treatment = as.logical(treatment),
    income_bracket = as.factor(income_bracket),
    region = as.factor(region))

## Specify a causal question

In [ ]:
df |>
    ggplot(aes(x = churn_risk_score, fill = treatment)) +
    geom_density()

In [ ]:
df |>
  group_by(treatment) |> 
  summarize(churn_risk_score = mean(churn_risk_score), .groups = 'drop')

In [ ]:
library(broom)
df |>
  lm(churn_risk_score ~ treatment, data = _) |>
  tidy()

## Draw our assumptions using a causal diagram (DAG)

In [ ]:
library(dagitty)

dag <- dagitty('dag {
  Age -> PaymentHistory
  IncomeBracket -> PaymentHistory
  Region -> MonthlyConsumption
  HouseholdSize -> MonthlyConsumption
  Age -> Satisfaction
  IncomeBracket -> Satisfaction
  Age -> Complaints
  IncomeBracket -> Complaints

  PaymentHistory -> ChurnRiskScore
  MonthlyConsumption -> ChurnRiskScore
  Satisfaction -> ChurnRiskScore
  Complaints -> ChurnRiskScore

  ChurnRiskScore -> flag_unsatisfied
  Satisfaction -> flag_unsatisfied

  ChurnRiskScore -> Treatment
  flag_unsatisfied -> Treatment

  ChurnRiskScore -> Churned
  Treatment -> Churned
}')

# Visualizamos
plot(dag)

## Model our assumptions

### Propensity model

In [ ]:
the_formula <- formula("treatment ~ age + income_bracket + region + household_size + tenure + payment_history + monthly_consumption + customer_satisfaction_score + complaints_last_year")
# the_formula <- formula(treatment ~ churn_risk_score)

propensity_model <- glm(
  the_formula,
  data = df,
  family = binomial()
)

In [ ]:
library(broom)
library(propensity)

df_wts <- propensity_model |>
  augment(data = df, type.predict = "response") |>
  # .fitted is the value predicted by the model
  # for a given observation
  mutate(wts = wt_ate(.fitted, treatment))

df_wts |>
  select(treatment, .fitted, wts) |>
  head()

## Diagnose our models

<!-- ## Diagnose our models -->

In [ ]:
library(halfmoon)
ggplot(df_wts, aes(.fitted)) +
  geom_mirror_histogram(
    aes(fill = treatment),
    bins = 50
  ) +
  scale_y_continuous(labels = abs) +
  labs(x = "propensity score")

In [ ]:
ggplot(df_wts, aes(.fitted)) +
  geom_mirror_histogram(
    aes(group = treatment),
    bins = 50
  ) +
  geom_mirror_histogram(
    aes(fill = treatment, weight = wts),
    bins = 50,
    alpha = .5
  ) +
  scale_y_continuous(labels = abs) +
  labs(x = "propensity score")

In [ ]:
plot_df <- tidy_smd(
  df_wts,
  c(income_bracket, region, household_size, tenure, payment_history, monthly_consumption, customer_satisfaction_score, complaints_last_year),
  # c(churn_risk_score),
  .group = treatment,
  .wts = wts
)

ggplot(
  plot_df,
  aes(
    x = abs(smd),
    y = variable,
    group = method,
    color = method
  )
) +
  geom_love()

In [ ]:
df_wts |>
  ggplot(aes(wts)) +
  geom_density(fill = "#CC79A7", color = NA, alpha = 0.8)

## Estimate the causal effect

### Using the weights

In [ ]:
df_wts |>
  lm(churned ~ treatment, data = _, weights = wts) |>
  tidy(conf.int = TRUE)

In [ ]:
library(rsample)

fit_ipw <- function(.split, ...) {
  # get bootstrapped data frame
  .df <- as.data.frame(.split)

  # fit propensity score model
  propensity_model <- glm(
    the_formula,
    data = .df,
    family = binomial()
  )

  # calculate inverse probability weights
  .df <- propensity_model |>
    augment(type.predict = "response", data = .df) |>
    mutate(wts = wt_ate(.fitted, treatment))

  # fit correctly bootstrapped ipw model
  lm(churned ~ treatment, data = .df, weights = wts) |>
    tidy()
}

In [ ]:
bootstrapped_net_data <- bootstraps(
  df,
  times = 1000,
  # required to calculate CIs later
  apparent = TRUE
)

bootstrapped_net_data

In [ ]:
ipw_results <- bootstrapped_net_data |>
  mutate(boot_fits = map(splits, fit_ipw))

ipw_results

In [ ]:
ipw_results$boot_fits |> head()

In [ ]:
ipw_results$boot_fits[[1]]

In [ ]:
ipw_results |>
  # remove original data set results
  filter(id != "Apparent") |> 
  mutate(
    estimate = map_dbl(
      boot_fits,
      # pull the `estimate` for `netTRUE` for each fit
      \(.fit) .fit |>
        filter(term == "treatmentTRUE") |>
        pull(estimate)
    )
  ) |>
  ggplot(aes(estimate)) +
  geom_histogram(fill = "#D55E00FF", color = "white", alpha = 0.8)

In [ ]:
boot_estimate <- ipw_results |>
  # calculate T-statistic-based CIs
  int_t(boot_fits) |>
  filter(term == "treatmentTRUE")

boot_estimate

### Directly

Para ver que, sin pesos, los subestima ligeramente

In [ ]:
# Fit a logistic regression model Churned ~ Treatment
log_model <- glm(churned ~ treatment, data = df, family = "binomial")
summary(log_model)

In [ ]:
as.numeric(1 + exp(-coef(log_model)['(Intercept)'])) / as.numeric(1 + exp(-coef(log_model)['(Intercept)']-coef(log_model)['treatmentTRUE']))

In [ ]:
lm_model <- lm(churn_risk_score ~ treatment, data = df)
summary(lm_model)

In [ ]:
confint(lm_model)['treatmentTRUE',]

In [ ]:
lm_model |>
  tidy(conf.int = TRUE)

## Conduct sensitivity analysis on the effect estimate